# Analysing detection of faces with trackers

## Importing libraries

In [1]:
import gc
import os
import sys
import math
import glob
import tqdm
import random
import numpy as np
from tqdm import tqdm
import time
from time import sleep

In [2]:
import pandas as pd
import xml.etree.ElementTree as ET

In [3]:
import cv2
import dlib
from imutils import face_utils

## Defining Paths

In [4]:
path_folder_videos = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/all/'

In [5]:
path_folder_of_folders = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/'

In [6]:
path_xml_dlib_output = 'output_XML/dlib_hog_output_232_folders.xml'

In [7]:
path_sample_video = path_folder_videos + "VIDEO_male-delight2_20180517_111853_1366476353.mp4"

In [8]:
path_sample_video_folder = path_folder_of_folders + "VIDEO_male-delight2_20180517_111853_1366476353 (5-27-2018 5-44-26 AM)/"

In [9]:
path_sample_video_folder2 = path_folder_of_folders + "VIDEO_1526475571181 bored_20180516_103925_356651116 (5-29-2018 10-06-12 AM)"
path_sample_video_folder2

'/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_1526475571181 bored_20180516_103925_356651116 (5-29-2018 10-06-12 AM)'

## Reading the XML file and getting the boxes and frame numbers in which face is detected

In [10]:
def getDictVideoToFrameNoBoxFromXML(path_xml_dlib_output):
    """
    Returns the dictionary {Video_name: {frame_1:[box_1 ie l,t,w,h], frame_2:[box_2, box_3]}} from XML file.
    """
    
    dictBox = {}
    
    #read the XML file
    tree = ET.parse(path_xml_dlib_output)
    root = tree.getroot()
    
    for image in tqdm(root.iter('image')):
        name_file = image.attrib['file']
        name_video = name_file.split('.')[0].rsplit(' ',1)[0]
        num_frame = (int)(name_file.split('.')[0].rsplit(' ',1)[1])
        
        list_boxes = []
        if (len(image) == 0):
            list_boxes = []
        else:
            for box_num,box in enumerate(image):
                box_attribs = box.attrib
                box_data = [box_attribs['left'], box_attribs['top'],box_attribs['width'],box_attribs['height']]
                list_boxes.append(box_data)
        
        if name_video in dictBox:
            dictBox[name_video][num_frame] = list_boxes
        else:
            dictBox[name_video] = {num_frame : list_boxes}
            
    return dictBox
        
        

## Detecting faces

In [11]:
def detectFaceFromPath(impath, detector):
    """
    Given the path of an image, and the detector, reads the image in grayscale and returns the bounding box rectangle
    in the format [(x,y,w,h)]
    """
    
    # load the input image, resize it, and convert it to grayscale
    image = cv2.imread(impath,0)
    
    # detect faces in the grayscale image
    rects = detector(gray, 1)
    
    result = []
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
#         shape = predictor(gray, rect)
#         shape = face_utils.shape_to_np(shape)
#         print(shape.shape)
        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        result.append((x,y,w,h))
    
    return result
#         cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # show the face number
#         cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
#         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
#         basePath=imageTouse.split('.')[0]
#         lmpath=basePath+"_landmarks"+str(i+1)+".txt"
#         text_file = open(lmpath, "w")
#         for (x, y) in shape:
#             cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
#             text_file.write(str(x)+' '+str(y)+'\n')
#         text_file.close()


In [12]:
def detectFaceFromGrayImage(gray_image, detector):
    """
    Given the gray_image, and the detector, reads the image in grayscale and returns the bounding box rectangle
    in the format [(x,y,w,h)]
    """
    # detect faces in the grayscale image
    rects = detector(gray_image, 1)
    
    result = []
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
#         shape = predictor(gray, rect)
#         shape = face_utils.shape_to_np(shape)
#         print(shape.shape)
        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        result.append((x,y,w,h))
    
    return result

## Understanding working of trackers in OpenCV


Useful websites and places to find codebases:
    - [https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/] (https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/)
    - [https://www.learnopencv.com/object-tracking-using-opencv-cpp-python/] (https://www.learnopencv.com/object-tracking-using-opencv-cpp-python/)

Code from link 2 (find learnopencv repo on github and locally at ~):
    
    Do init for a frame and then simply get the tracked box using tracker.update(frame)
    
Code from link 1 (adrian pyimagesearch):
    
    

Observation: KCF tracker is working out to be well.

Get the labels for detected faces in some frames of the video -> get accuracy
- Identify the frames in succession in which faces are not detected, and use the previously labelled frames to make new predictions -> get accuracy again
- Identify the frames in succession in which faces are not detected, take a window, and detect faces again
- Identify the frames in succession in which faces are not detected, take a window, and detect faces again after normalising that window

In [12]:
def processVideo(path_video_frames_folder, dictBox):
    """
    Get dictionary of the form {Video_name: {frame_1:[box_1 ie l,t,w,h], frame_2:[box_2, box_3]}} 
    after updating with tracked boxes (for a single face now.)    
    """
    
    #initiate the tracker
    tracker = cv2.TrackerKCF_create()
    
    # get name of video
    name_video = os.path.basename(path_video_frames_folder).rsplit(' ',3)[0]
#     print(name_video)
    # get dict frame no: box
    
    dictFrameNoToBoxes = dictBox[name_video]
    tracking = False
    
    # when frame no comes,check if detection(dict[frame_no] has a box): if not,track and get a box 
        # Save this box
        # Detect face inside the box obtained after resizing this box
            # Save this box
        # normalise this window and detect face.
    
    # Get path of all images in this video and sort them, so that video frames are read in order
    list_image_files = glob.glob(path_video_frames_folder + '/*')
    list_image_files.sort()
    
    count_tracked_frames = 0
    count_total_frames = len(list_image_files)
    print(len(list_image_files),len(dictFrameNoToBoxes))
    for frame_no, path_file in enumerate(list_image_files,start=1):
        
#         print("Starting frame number ", frame_no)
        #read the image in grayscale
        frame = cv2.imread(path_file,0)
        
        #list of boxes eg - [[l,t,w,h], [l,t,w,h]]
        if frame_no in dictFrameNoToBoxes:
            boxes_in_this_frame = dictFrameNoToBoxes[frame_no]
        else:
            boxes_in_this_frame = []
#         print("Boxes in this frame", boxes_in_this_frame)
        
        # if the frame has atleast 1 box detected
        if len(boxes_in_this_frame) > 0:
            
            ### check and mark the relevant box, convert to float as reqd by init
            # adjust for multiple faces.
            box_main = (boxes_in_this_frame[0])
            box_main = tuple([float(i) for i in box_main])
            
            # reinitialise the tracker:
            ok = tracker.init(frame, box_main)
            tracking = True
        
        # else - when no boxes detected 
        else:
            # if tracking, update
            if tracking:
                ok, tracked_box = tracker.update(frame)
                print("Tracking", ok, tracked_box)
                
                #writing to dict
                tracked_box_int = list(int(i) for i in tracked_box)
                dictBox[name_video][frame_no] = [tracked_box_int]
                count_tracked_frames += 1
                
            # if not tracking, don't update
            else:
#                 print("Not Tracking")
                pass
    
    print("Tracked {}/{} frames in video: {}".format(count_tracked_frames, count_total_frames, name_video))
    return dictBox

#sample function to try
def tryProcessVideo():
    dict_box = getDictVideoToFrameNoBoxFromXML(path_xml_dlib_output)
    print(len(dict_box))
    processVideo(path_sample_video_folder2, dict_box)
tryProcessVideo()

39016it [00:00, 150958.56it/s]


233
(228, 228)
('Tracking', True, (1170.0, 92.0, 462.0, 462.0))
('Tracking', True, (1208.0, 70.0, 462.0, 462.0))
('Tracking', True, (1232.0, 56.0, 462.0, 462.0))
('Tracking', True, (1210.0, 130.0, 462.0, 462.0))
('Tracking', True, (1182.0, 162.0, 462.0, 462.0))
('Tracking', True, (1156.0, 114.0, 462.0, 462.0))
('Tracking', True, (1166.0, 84.0, 462.0, 462.0))
('Tracking', True, (1176.0, 56.0, 462.0, 462.0))
('Tracking', True, (1184.0, 26.0, 462.0, 462.0))
('Tracking', True, (1186.0, -2.0, 462.0, 462.0))
('Tracking', True, (1184.0, -26.0, 462.0, 462.0))
('Tracking', True, (1186.0, -38.0, 462.0, 462.0))
('Tracking', True, (1220.0, -4.0, 462.0, 462.0))
('Tracking', True, (1228.0, -24.0, 462.0, 462.0))
('Tracking', True, (1226.0, -40.0, 462.0, 462.0))
('Tracking', True, (1214.0, -42.0, 462.0, 462.0))
('Tracking', True, (1206.0, -30.0, 462.0, 462.0))
('Tracking', True, (1208.0, -8.0, 462.0, 462.0))
('Tracking', True, (1232.0, -34.0, 462.0, 462.0))
('Tracking', True, (1246.0, -42.0, 462.0, 46

### Multiple subjects

#### Code for rectangle intersection

In [13]:
def box_percent_covered (a,b):
    inner = a.intersect(b).area()
    if inner == 0:
        return 0
    return max(inner/a.area(), inner/b.area())

def drect_add(a,b):
    """
    Being able to add two drectangles, as addition is not available in the version of dlib in anaconda environment.
    """
    
    return dlib.drectangle(min(a.left(),b.left()),min(a.top(),b.top()),max(a.right(),b.right()),max(a.bottom(),b.bottom()))

def intersect(a,b,iou_thresh=0.5,percent_covered_thresh=1.0):
    """Determines if two dlib rects intersect or not"""
    inner = a.intersect(b).area()
    if (inner == 0):
        return False
    
    outer = drect_add(a,b).area()
    print("inner,outer,a.area,b.area: ", inner, outer, a.area(),b.area())
    if (inner/outer > iou_thresh or 
        inner/a.area() > percent_covered_thresh or 
        inner/b.area() > percent_covered_thresh):
        return True
    else:
        return False

#### Code for updating trackers

In [14]:
def getUpdatedTrackingBoxStatus(boxes_in_this_frame, list_tracked_boxes):
    """
    Returns the boxes status ie which boxes are 
    matched: (need to be updated), 
    tracked_not_detected: unmatched(no new detections) and in tracker(ie need to be updated), 
    detected_not_tracked: unmatched and not tracked(new detections, new trackers have to be initialised)
    """
    
    # initialise the things to be returned
    dict_tracked_to_detected = {}
    list_detected_not_tracked = []
    list_tracked_not_detected = []
    
    # see the matched status for both, tracked boxes and detected boxes
    status_tracked_boxes = [False] * len(list_tracked_boxes) # marks if box is considered or not
    status_detected_boxes_in_frame = [False] * len(boxes_in_this_frame)
    
    # see the intersection of each tracked box with each detected box (if not already matched).
    for num_tracked_box,tracked_box in enumerate(list_tracked_boxes):
        # match current tracked box to each, change status to True if matched
        for num_detected_box,detected_box in enumerate(boxes_in_this_frame):
            rect_tracked = dlib.drectangle(tracked_box)
            rect_detected = dlib.drectangle(detected_box)
            # if box is not already assigned, check for intersection and update matched status
            if not status_detected_boxes_in_frame[num_detected_box]:
                bool_intersect = intersect(tracked_box,detected_box)
                if bool_intersect:
                    status_tracked_boxes[num_tracked_box] = True
                    status_detected_boxes_in_frame[num_detected_box] = True
                    dict_tracked_to_detected[num_tracked_box] = num_detected_box
    
    # populate list_detected_not_tracked and tracked_not_detected
    for num_detected_box, status_detected in enumerate(status_detected_boxes_in_frame):
        if not status_detected:
            list_detected_not_tracked.append (num_detected_box)
    for num_tracked_box, status_tracked in enumerate(status_tracked_boxes):
        if not status_tracked:
            list_tracked_not_detected.append(num_tracked_box)
    
    return  dict_tracked_to_detected, list_detected_not_tracked, list_tracked_not_detected

In [44]:
def processVideoMultipleObjects(path_video_frames_folder, dictBox):
    """
    Track multiple faces in a video.
    """
    
#     Initiate the trackers list
    list_trackers = []
#     tracker = cv2.TrackerKCF_create()
    
    # get name of video
    name_video = os.path.basename(path_sample_video_folder2).rsplit(' ',3)[0]
    print(name_video)
    
    # get dict -> {frame no: box}
    dictFrameNoToBoxes = dictBox[name_video]
#     print(dictFrameNoToBoxes)
    
    tracking = False
    # when frame no comes,check if detection(dict[frame_no] has a box): if not,track and get a box 
        # Save this box
        # Detect face inside the box obtained after resizing this box
            # Save this box
        # normalise this window and detect face.
    
    # Get path of all images in this video
    list_image_files = glob.glob(path_video_frames_folder + '/*')
    list_image_files.sort()
    print(len(list_image_files))
    
    for frame_no, path_file in enumerate(list_image_files,start=1):
        
        print("Starting frame number ", frame_no)
        # read image
        frame = cv2.imread(path_file,0)
        
        if frame_no in dictFrameNoToBoxes:
            boxes_in_this_frame = dictFrameNoToBoxes[frame_no]
        else:
            boxes_in_this_frame = []
            
        # list of boxes eg - [[l,t,w,h], [l,t,w,h]]
#         print(len(dictFrameNoToBoxes),frame_no)
#         boxes_in_this_frame = dictFrameNoToBoxes[frame_no]
        print("Detected boxes in this frame ", boxes_in_this_frame)
        
        # if the frame has a box detected, reinitialise the tracker
        if len(boxes_in_this_frame) > 0:
            
            # match to find out all the boxes which are already being tracked
            ## ie match the boxes detected (in boxes_in_this_frame) with the boxes in each tracker,            
            ### For the ones matched, init with new labels(boxes_in_this_frame).
            ### For unmatched and new labels, init.
            ### For unmatched and tracked, continue tracking 
            list_tracked_boxes = []
            for tracker in list_trackers:
                ok, box = tracker.update(frame) # remember that the tracked box returns (0.0,0.0,0.0,0.0) if not ok.
                list_tracked_boxes.append(box)
            print("Tracked boxes in this frame ", list_tracked_boxes)
            list_index_detected_box_matched, list_detected_not_tracked, list_tracked_not_detected = getUpdatedTrackingBoxStatus(boxes_in_this_frame, list_tracked_boxes)
#             list_trackers = getUpdatedDictBoxFromXML(boxes_in_this_frame, list_tracked_boxes)
            print("list_index_detected_box_matched, list_detected_not_tracked, list_tracked_not_detected", list_index_detected_box_matched, list_detected_not_tracked, list_tracked_not_detected )
            
            # For the ones matched, init with new labels(boxes_in_this_frame)
            dict_tracked_to_detected = 
            list_index_tracked = dict_tracked_to_detected.keys()
            for i in list_index_tracked:
                
                box_main = boxes_in_this_frame[dict_tracked_to_detected[i]]
                box_detected = tuple([float(j) for j in box_main])
                ok = list_trackers[i].init(frame,box_detected)
                
            ### For unmatched and tracked, continue tracking 
            for i in list_tracked_not_detected:
                ok, tracked_box = list_trackers[i].update(frame)
                
            ### For unmatched and new labels, init
            for i in list_detected_not_tracked:
                tracker = cv2.TrackerKCF_create()
                box_detected = boxes_in_this_frame[i]
                box_detected = tuple([float(j) for j in box_detected])
                ok = tracker.init(frame,box_detected)
                list_trackers.append(tracker)
            
            ## check and mark the relevant box, convert to float as reqd by init
#             box_main = (boxes_in_this_frame[0])
#             box_main = tuple([float(i) for i in box_main])
            
            # reinitialise the tracker:
#             ok = tracker.init(frame, box_main)
#             tracking = True
        
#         else - when no boxes detected, track and update the bounding box
        else:
            print("in else")
            # if tracking, update all trackers.
            if tracking:
                print("Tracking")
                for tracker in list_trackers:
                    ok, tracked_box = tracker.update(frame)
                    print("0 boxes, tracking", ok, tracked_box)
                
                #writing to dict
                tracked_box_int = tuple(int(i) for i in tracked_box)
                print("int",tracked_box_int)
            # if not tracking, don't update
            else:
                print("Not Tracking")

def tryProcessVideo():
    dict_box = getDictVideoToFrameNoBoxFromXML(path_xml_dlib_output)
    processVideoMultipleObjects(path_sample_video_folder2, dict_box)
tryProcessVideo()

39016it [00:00, 128689.64it/s]

VIDEO_1526475571181 bored_20180516_103925_356651116
228
('Starting frame number ', 1)
(228, 1)
('Detected boxes in this frame ', [['1170', '92', '462', '462']])
('Tracked boxes in this frame ', [])
('list_index_detected_box_matched, list_detected_not_tracked, list_tracked_not_detected', [], [0], [])
('Starting frame number ', 2)
(228, 2)
('Detected boxes in this frame ', [['1170', '92', '462', '462']])
('Tracked boxes in this frame ', [])
('list_index_detected_box_matched, list_detected_not_tracked, list_tracked_not_detected', [], [0], [])
('Starting frame number ', 3)
(228, 3)
('Detected boxes in this frame ', [])
in else
Not Tracking
('Starting frame number ', 4)
(228, 4)
('Detected boxes in this frame ', [['1489', '770', '36', '36']])
('Tracked boxes in this frame ', [])
('list_index_detected_box_matched, list_detected_not_tracked, list_tracked_not_detected', [], [0], [])
('Starting frame number ', 5)
(228, 5)
('Detected boxes in this frame ', [])
in else
Not Tracking
('Starting fra

In [39]:
processVideo(path_sample_video_folder2,dictBox)

Tracked 0/228 frames in video: VIDEO_1526475571181 bored_20180516_103925_356651116


{'VIDEO_00-male-surprise_20180516_091529_219393302': {1: [['-137',
    '295',
    '616',
    '555'],
   ['1060', '675', '385', '385']],
  2: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  3: [['1016', '605', '462', '462'], ['-63', '348', '514', '463']],
  4: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  5: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  6: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  7: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  8: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  9: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  10: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  11: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  12: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  13: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  14: [['-63', '348', '514', '463'], ['1

In [40]:
processVideoMultipleObjects(path_sample_video_folder2,dictBox)

VIDEO_1526475571181 bored_20180516_103925_356651116
369
('Starting frame number ', 1)
(228, 1)
('Detected boxes in this frame ', [['1170', '92', '462', '462']])
('Tracked boxes in this frame ', [])
('list_index_detected_box_matched, list_detected_not_tracked, list_tracked_not_detected', [], [0], [])
('Starting frame number ', 2)
(228, 2)
('Detected boxes in this frame ', [['1170', '92', '462', '462']])
('Tracked boxes in this frame ', [])
('list_index_detected_box_matched, list_detected_not_tracked, list_tracked_not_detected', [], [0], [])
('Starting frame number ', 3)
(228, 3)
('Detected boxes in this frame ', [[1170, 92, 462, 462]])
('Tracked boxes in this frame ', [])
('list_index_detected_box_matched, list_detected_not_tracked, list_tracked_not_detected', [], [0], [])
('Starting frame number ', 4)
(228, 4)
('Detected boxes in this frame ', [['1489', '770', '36', '36']])
('Tracked boxes in this frame ', [])
('list_index_detected_box_matched, list_detected_not_tracked, list_tracked_n

KeyError: 229

### Update dictionary for all videos

In [48]:
def getUpdatedDictBoxFromXML(path_xml, path_folder_of_folders):
    """
    Get updated dictionary of the form {Video_name: {frame_1:[box_1 ie l,t,w,h], frame_2:[box_2, box_3]}} 
    after tracking frames in *all* the videos.
    
    Parameters
    ----------
    path_xml : str
        Path to the original XML (with the frame rectangles detected by a detector)
        
    path_folder_of_folders: str
        Path to the folder having folders of Video frames.
        
    Returns
    -------
    dictionary of dictionary of lists
        Returns the dictionary of the form {Video_name: {frame_1:[box_1 ie l,t,w,h], frame_2:[box_2, box_3]}}  
        with the updated boxes after tracking.
        
    """
    
    updated_dict_box = getDictVideoToFrameNoBoxFromXML(path_xml)
    list_video_names = updated_dict_box.keys()
    list_path_folder_video_frames = glob.glob(path_folder_of_folders + "/*")
    list_path_video_folder = []
    
    # Make a list of path of video folders which appear in the dictionary.
    for video_name in list_video_names:
        for path_folder in list_path_folder_video_frames:
            if video_name in path_folder:
                list_path_video_folder.append(path_folder)
                break
    
    print(len(list_path_video_folder), ": number of folders are being looked at")

    # Update the dictionary after processing all the videos
    for path_video_folder in tqdm(list_path_video_folder):
#         print(path_video_folder)
        updated_dict_box = processVideo(path_video_folder,updated_dict_box)
    
    return updated_dict_box

In [34]:
final_dict_box = getUpdatedDictBoxFromXML(path_xml_dlib_output,path_folder_of_folders)

39016it [00:00, 146206.22it/s]
  0%|          | 0/233 [00:00<?, ?it/s]

(233, ': number of folders are being looked at')


  0%|          | 1/233 [00:02<11:29,  2.97s/it]

Tracked 0/84 frames in video: VIDEO_1526475571181 bored_20180516_104528_1803080270
('Tracking', True, (1281.0, 418.0, 554.0, 555.0))
('Tracking', False, (0.0, 0.0, 0.0, 0.0))
('Tracking', False, (0.0, 0.0, 0.0, 0.0))
('Tracking', False, (0.0, 0.0, 0.0, 0.0))


  1%|          | 2/233 [00:06<12:54,  3.35s/it]

Tracked 4/140 frames in video: VIDEO_1526475571181 bored_20180516_110804_731160236


  1%|▏         | 3/233 [00:08<10:44,  2.80s/it]

('Tracking', True, (676.0, 439.0, 267.0, 267.0))
('Tracking', True, (679.0, 438.0, 267.0, 267.0))
('Tracking', False, (0.0, 0.0, 0.0, 0.0))
('Tracking', False, (0.0, 0.0, 0.0, 0.0))
('Tracking', False, (0.0, 0.0, 0.0, 0.0))
('Tracking', False, (0.0, 0.0, 0.0, 0.0))
('Tracking', True, (676.0, 439.0, 267.0, 267.0))
Tracked 7/153 frames in video: VIDEO_10-male-confusion_20180517_050147_1758874434


KeyboardInterrupt: 

### Write a new XML 

In [ ]:
def writeXMLFromDictBox(updated_dict_box, path_folder_destination):
    """
    Takes the dictionary and saves an XML file at given destination folder for analysis
    Currently saving after each video for backup.
    """
    
    root = ET.Element("dataset")
    name = ET.SubElement(root, "name").text = "Labelled faces"
    comment = ET.SubElement(root, "comment").text = "These are labelled images from Bagamoyo"
    images = ET.SubElement(root, "images")
    
    # make a filename for each video name
    for video_name in tqdm(updated_dict_box.keys()):
        dict_framenum_to_listboxes = updated_dict_box[video_name]
        frame_numbers = dict_framenum_to_listboxes.keys()
        frame_numbers.sort()
        for frame_num in frame_numbers:
#             xml append image
            image_file_name = video_name + " " + "{0:0=3d}".format(frame_num)
            image_node = ET.SubElement(images, "image", file=image_file_name)
            for box in dict_framenum_to_listboxes[frame_num]:
#                 create box, add to 
                l,t,w,h = box[0],box[1],box[2],box[3]
                box = ET.SubElement(image_node, "box", height=str(h), left=str(l), top=str(t), width=str(w))
    
        tree = ET.ElementTree(root)
        dest_name = path_folder_destination + video_name + '.xml'
        tree.write(dest_name)
        print("xml written to {}".format(dest_name))

In [17]:
writeXMLFromDictBox(final_dict_box,'output_XML/tracking/')

  3%|▎         | 7/233 [00:00<00:10, 21.73it/s]

xml written to output_XML/tracking/VIDEO_1526475571181 bored_20180516_104528_1803080270.xml
xml written to output_XML/tracking/VIDEO_1526475571181 bored_20180516_110804_731160236.xml
xml written to output_XML/tracking/VIDEO_10-male-confusion_20180517_050147_1758874434.xml
xml written to output_XML/tracking/VIDEO_1526469405985 make a face like this one_20180516_105329_242798671.xml
xml written to output_XML/tracking/VIDEO_06-female-confusion_20180516_110221_101963203.xml
xml written to output_XML/tracking/VIDEO_1526469405985 make a face like this one_20180516_110124_368117096.xml
xml written to output_XML/tracking/VIDEO_03-female-delight_20180516_104912_855688121.xml
xml written to output_XML/tracking/VIDEO_1526469405985 make a face like this one_20180517_051046_318795747.xml
xml written to output_XML/tracking/VIDEO_1526475571181 bored_20180516_105909_1662154728.xml
xml written to output_XML/tracking/VIDEO_00-male-surprise_20180516_091551_1272027070.xml
xml written to output_XML/trackin

  7%|▋         | 16/233 [00:00<00:07, 29.31it/s]

xml written to output_XML/tracking/VIDEO_03-female-delight_20180516_102820_35617256.xml
xml written to output_XML/tracking/VIDEO_10-male-confusion_20180516_091825_514567706.xml
xml written to output_XML/tracking/VIDEO_1526475586855 confused_20180516_104514_1700391713.xml
xml written to output_XML/tracking/VIDEO_08-female-frustration_20180516_110052_2081001093.xml
xml written to output_XML/tracking/VIDEO_00-male-surprise_20180516_091859_1464474503.xml
xml written to output_XML/tracking/VIDEO_1526475571181 bored_20180516_105424_1564488712.xml
xml written to output_XML/tracking/VIDEO_03-female-delight_20180516_110715_1869255788.xml


  9%|▉         | 22/233 [00:00<00:07, 27.97it/s]

...

xml written to output_XML/tracking/VIDEO_1526475586855 confused_20180516_110810_1142721593.xml


100%|█████████▉| 232/233 [01:12<00:00,  3.20it/s]

xml written to output_XML/tracking/VIDEO_00-male-surprise_20180516_091541_1754978908.xml


100%|██████████| 233/233 [01:13<00:00,  3.18it/s]

xml written to output_XML/tracking/VIDEO_1526469405985 make a face like this one_20180517_045849_1455237136.xml


### Main

In [14]:
# def main():
dictBox = getDictVideoToFrameNoBoxFromXML(path_xml_dlib_output)

39016it [00:00, 121492.53it/s]


In [19]:
path_sample_video_folder2

'/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_1526475571181 bored_20180516_103925_356651116 (5-29-2018 10-06-12 AM)'

In [161]:
processVideo(path_sample_video_folder2,dictBox)
# main()

VIDEO_1526475571181 bored_20180516_103925_356651116
369
('Starting frame number ', 1)
('Boxes in this frame', [['1170', '92', '462', '462']])
('Starting frame number ', 2)
('Boxes in this frame', [['1170', '92', '462', '462']])
('Starting frame number ', 3)
('Boxes in this frame', [[1170, 92, 462, 462]])
('Starting frame number ', 4)
('Boxes in this frame', [['1489', '770', '36', '36']])
('Starting frame number ', 5)
('Boxes in this frame', [[1170, 92, 462, 462]])
('Starting frame number ', 6)
('Boxes in this frame', [[1170, 92, 462, 462]])
('Starting frame number ', 7)
('Boxes in this frame', [['1272', '40', '462', '462']])
('Starting frame number ', 8)
('Boxes in this frame', [['204', '333', '385', '385']])
('Starting frame number ', 9)
('Boxes in this frame', [['1324', '40', '462', '462'], ['247', '333', '385', '385']])
('Starting frame number ', 10)
('Boxes in this frame', [['1324', '40', '462', '462']])
('Starting frame number ', 11)
('Boxes in this frame', [['1375', '40', '462', 

('Boxes in this frame', [['1274', '33', '385', '385']])
('Starting frame number ', 95)
('Boxes in this frame', [['1272', '-62', '462', '513']])
('Starting frame number ', 96)
('Boxes in this frame', [['1272', '-62', '462', '513']])
('Starting frame number ', 97)
('Boxes in this frame', [['1272', '-62', '462', '513']])
('Starting frame number ', 98)
('Boxes in this frame', [[1172, 94, 462, 462]])
('Starting frame number ', 99)
('Boxes in this frame', [['1317', '-52', '385', '428']])
('Starting frame number ', 100)
('Boxes in this frame', [['161', '376', '385', '385'], ['1317', '-52', '385', '428']])
('Starting frame number ', 101)
('Boxes in this frame', [[1172, 92, 462, 462]])
('Starting frame number ', 102)
('Boxes in this frame', [['1324', '-62', '462', '513'], ['142', '348', '463', '463']])
('Starting frame number ', 103)
('Boxes in this frame', [[1172, 94, 462, 462]])
('Starting frame number ', 104)
('Boxes in this frame', [['204', '376', '385', '385']])
('Starting frame number ', 

('Boxes in this frame', [['1317', '33', '385', '385'], ['204', '461', '385', '385']])
('Starting frame number ', 175)
('Boxes in this frame', [['204', '461', '385', '385'], ['598', '669', '321', '321'], ['1317', '33', '385', '385']])
('Starting frame number ', 176)
('Boxes in this frame', [['204', '461', '385', '385'], ['1324', '40', '462', '462'], ['598', '669', '321', '321']])
('Starting frame number ', 177)
('Boxes in this frame', [['1317', '33', '385', '385']])
('Starting frame number ', 178)
('Boxes in this frame', [['1317', '33', '385', '385'], ['142', '400', '463', '462']])
('Starting frame number ', 179)
('Boxes in this frame', [['1317', '33', '385', '385']])
('Starting frame number ', 180)
('Boxes in this frame', [['1317', '33', '385', '385']])
('Starting frame number ', 181)
('Boxes in this frame', [['1272', '-62', '462', '513']])
('Starting frame number ', 182)
('Boxes in this frame', [['1272', '-62', '462', '513']])
('Starting frame number ', 183)
('Boxes in this frame', [[

('Tracking', True, (1174.0, 92.0, 462.0, 462.0))
('Starting frame number ', 260)
('Boxes in this frame', [])
('Tracking', True, (1172.0, 92.0, 462.0, 462.0))
('Starting frame number ', 261)
('Boxes in this frame', [])
('Tracking', True, (1174.0, 92.0, 462.0, 462.0))
('Starting frame number ', 262)
('Boxes in this frame', [])
('Tracking', True, (1172.0, 92.0, 462.0, 462.0))
('Starting frame number ', 263)
('Boxes in this frame', [])
('Tracking', True, (1174.0, 92.0, 462.0, 462.0))
('Starting frame number ', 264)
('Boxes in this frame', [])
('Tracking', True, (1172.0, 92.0, 462.0, 462.0))
('Starting frame number ', 265)
('Boxes in this frame', [])
('Tracking', True, (1174.0, 92.0, 462.0, 462.0))
('Starting frame number ', 266)
('Boxes in this frame', [])
('Tracking', True, (1172.0, 92.0, 462.0, 462.0))
('Starting frame number ', 267)
('Boxes in this frame', [])
('Tracking', True, (1174.0, 92.0, 462.0, 462.0))
('Starting frame number ', 268)
('Boxes in this frame', [])
('Tracking', True, 

('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 340)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 341)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 342)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 343)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 344)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 345)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 346)
('Boxes in this frame', [])
('Tracking', True, (1192.0, 88.0, 462.0, 462.0))
('Starting frame number ', 347)
('Boxes in this frame', [])
('Tracking', True, (1194.0, 88.0, 462.0, 462.0))
('Starting frame number ', 348)
('Boxes in this frame', [])
('Tracking', True, 

{'VIDEO_00-male-surprise_20180516_091529_219393302': {1: [['-137',
    '295',
    '616',
    '555'],
   ['1060', '675', '385', '385']],
  2: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  3: [['1016', '605', '462', '462'], ['-63', '348', '514', '463']],
  4: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  5: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  6: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  7: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  8: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  9: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  10: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  11: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  12: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  13: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  14: [['-63', '348', '514', '463'], ['1

In [136]:
dictBox['VIDEO_1526475571181 bored_20180516_103925_356651116']

{1: [['1170', '92', '462', '462']],
 2: [['1170', '92', '462', '462']],
 3: [],
 4: [['1489', '770', '36', '36']],
 5: [],
 6: [],
 7: [['1272', '40', '462', '462']],
 8: [['204', '333', '385', '385']],
 9: [['1324', '40', '462', '462'], ['247', '333', '385', '385']],
 10: [['1324', '40', '462', '462']],
 11: [['1375', '40', '462', '462'], ['290', '376', '385', '385']],
 12: [['1375', '40', '462', '462'], ['332', '418', '386', '386']],
 13: [['1375', '40', '462', '462'], ['332', '418', '386', '386']],
 14: [['290', '418', '385', '386']],
 15: [['290', '418', '385', '386']],
 16: [['290', '418', '385', '386'], ['1324', '40', '462', '462']],
 17: [['1324', '40', '462', '462'], ['247', '418', '385', '386']],
 18: [['247', '418', '385', '386'], ['1281', '49', '554', '554']],
 19: [['1272', '92', '462', '462'], ['194', '400', '462', '462']],
 20: [['1272', '92', '462', '462']],
 21: [['1272', '92', '462', '462'], ['204', '461', '385', '385']],
 22: [['204', '461', '385', '385'], ['1272', '9

# Debug

In [11]:
processVideo(path_folder_videos + "VIDEO_male-delight2_20180517_111853_1366476353.mp4",1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171


In [12]:
cap = cv2.VideoCapture(path_sample_video)
property_id = int(cv2.CAP_PROP_FRAME_COUNT) 
length = int(cv2.VideoCapture.get(cap, property_id))
print( length )

173


In [ ]:
path_folder_videos